In [ ]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import pymysql
from tkinter import *
from PIL import ImageTk

def connect_db():
    try:
        con = pymysql.connect(host='localhost',
                              user='root',
                              password='Khushboo1286',
                              database='recipe_manager')
        return con
    except pymysql.Error as e:
        messagebox.showerror("Error", f"Database connection error: {e}")
        return None


def add_recipe():
    title = simpledialog.askstring("Add Recipe", "Enter Recipe Title:")
    if title:
        ingredients = simpledialog.askstring("Add Recipe", "Enter Ingredients:")
        if ingredients:
            instructions = simpledialog.askstring("Add Recipe", "Enter Instructions:")
            if instructions:
                con = connect_db()
                if con:
                    try:
                        with con.cursor() as cursor:
                            sql = "INSERT INTO recipes (title, ingredients, instructions) VALUES (%s, %s, %s)"
                            cursor.execute(sql, (title, ingredients, instructions))
                            con.commit()
                            messagebox.showinfo("Success", "Recipe added successfully!")
                            display_recipes()  # Refresh displayed recipes after adding
                    except pymysql.Error as e:
                        messagebox.showerror("Error", f"Failed to add recipe: {e}")
                    finally:
                        con.close()

# Function to view all recipes from the database
def view_recipes():
    con = connect_db()
    if con:
        try:
            with con.cursor() as cursor:
                sql = "SELECT * FROM recipes"
                cursor.execute(sql)
                recipes = cursor.fetchall()
                if recipes:
                    display_recipes_window(recipes)
                else:
                    messagebox.showinfo("Info", "No recipes found.")
        except pymysql.Error as e:
            messagebox.showerror("Error", f"Failed to fetch recipes: {e}")
        finally:
            con.close()

# Function to update a recipe in the database
def update_recipe():
    recipe_id = simpledialog.askinteger("Update Recipe", "Enter Recipe ID to Update:")
    if recipe_id:
        con = connect_db()
        if con:
            try:
                with con.cursor() as cursor:
                    # Check if recipe exists
                    cursor.execute("SELECT * FROM recipes WHERE recipe_id = %s", (recipe_id,))
                    recipe = cursor.fetchone()
                    if recipe:
                        title = simpledialog.askstring("Update Recipe", "Enter New Recipe Title:", initialvalue=recipe[1])
                        if title:
                            ingredients = simpledialog.askstring("Update Recipe", "Enter New Ingredients:", initialvalue=recipe[2])
                            if ingredients:
                                instructions = simpledialog.askstring("Update Recipe", "Enter New Instructions:", initialvalue=recipe[3])
                                if instructions:
                                    sql = "UPDATE recipes SET title = %s, ingredients = %s, instructions = %s WHERE recipe_id = %s"
                                    cursor.execute(sql, (title, ingredients, instructions, recipe_id))
                                    con.commit()
                                    messagebox.showinfo("Success", "Recipe updated successfully!")
                                    display_recipes()  # Refresh displayed recipes after updating
                    else:
                        messagebox.showerror("Error", "Recipe not found.")
            except pymysql.Error as e:
                messagebox.showerror("Error", f"Failed to update recipe: {e}")
            finally:
                con.close()

# Function to delete a recipe from the database
def delete_recipe():
    recipe_id = simpledialog.askinteger("Delete Recipe", "Enter Recipe ID to Delete:")
    if recipe_id:
        con = connect_db()
        if con:
            try:
                with con.cursor() as cursor:
                    # Check if recipe exists
                    cursor.execute("SELECT * FROM recipes WHERE recipe_id = %s", (recipe_id,))
                    recipe = cursor.fetchone()
                    if recipe:
                        confirm = messagebox.askyesno("Confirm Deletion", f"Are you sure you want to delete Recipe ID: {recipe_id}?")
                        if confirm:
                            sql = "DELETE FROM recipes WHERE recipe_id = %s"
                            cursor.execute(sql, (recipe_id,))
                            con.commit()
                            messagebox.showinfo("Success", "Recipe deleted successfully!")
                            display_recipes()  # Refresh displayed recipes after deleting
                    else:
                        messagebox.showerror("Error", "Recipe not found.")
            except pymysql.Error as e:
                messagebox.showerror("Error", f"Failed to delete recipe: {e}")
            finally:
                con.close()

# Function to display recipes in a separate window
def display_recipes_window(recipes):
    root = tk.Toplevel()
    root.title("Recipes")

    # Frame to hold recipe list
    frame = tk.Frame(root)
    frame.pack(padx=10, pady=10)

    # Scrollbar for recipe list
    scrollbar = tk.Scrollbar(frame, orient="vertical")
    scrollbar.pack(side="right", fill="y")

    # Listbox to display recipes
    recipe_listbox = tk.Listbox(frame, yscrollcommand=scrollbar.set, width=60, height=20)
    for recipe in recipes:
        recipe_listbox.insert(tk.END, f"ID: {recipe[0]} - {recipe[1]}")
    recipe_listbox.pack(side="left", fill="both")
    scrollbar.config(command=recipe_listbox.yview)

    # Button to close window
    close_button = tk.Button(root, text="Close", command=root.destroy)
    close_button.pack(pady=10)

# GUI setup
def main():
    root = tk.Tk()
    root.title("Recipe Manager")
    
    background=PhotoImage(file=r'C:\Users\khush\Downloads\background.png')
    bgLabel=Label(root,image=background)
    bgLabel.place(x=0,y=0)
    


    # Buttons for CRUD operations
    add_button = tk.Button(root, text="Add Recipe", command=add_recipe)
    add_button.grid(padx=700,pady=(250,1))

    view_button = tk.Button(root, text="View Recipes", command=view_recipes)
    view_button.grid(padx=680,pady=(140,1))

    update_button = tk.Button(root, text="Update Recipe", command=update_recipe)
    update_button.grid(padx=680,pady=(145,1))

    delete_button = tk.Button(root, text="Delete Recipe", command=delete_recipe)
    delete_button.grid( padx=680,pady=(100,1))

    root.mainloop()


if __name__ == "__main__":
    main()
